In [16]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import datetime, re, requests, io, time, random, string
from credentials import email, password
from list_of_stocks import stock_list

In [80]:
driver = webdriver.Chrome('/Users/alan/Web Development/General Assembly/SEI1019/unit-04/codealong/get-stock-data/chromedriver')
time.sleep(3)

In [81]:
def sign_in(email=email, password=password):
    driver.get('https://wallmine.com')
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click()
    time.sleep(0.1)
    if 'We\'re glad you\'re back!' in driver.page_source:
        print('On sign-in page.')
        driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
        driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
        time.sleep(0.1)
        driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
        time.sleep(3)
        if 'Stock market overview' in driver.page_source:
            print('Sign-in successful.')
    else:
        print('Restart app.')

In [82]:
sign_in(email, password)

On sign-in page.
Sign-in successful.


In [ ]:
def change_to_float(xpath):
    if xpath:
        string = driver.find_element_by_xpath(xpath).text
        elif string[0] == '$':
            if string[-1] == 'T':
                return float(string[1:-1]) * 1000000000000
            elif string[-1] == 'B':
                return float(string[1:-1]) * 1000000000
            elif string[-1] == 'M':
                return float(string[1:-1]) * 1000000
        else:
            if string[-1] == 'T':
                return float(string[:-1]) * 1000000000000
            elif string[-1] == 'B':
                return float(string[:-1]) * 1000000000
            elif string[-1] == 'M':
                return float(string[:-1]) * 1000000
    else:
        return 'N/A'


In [86]:
def get_data(stock_list):
    for i in range(len(stock_list)):
        stock = stock_list[i]
        print(f'{i}: {stock.get("stock_ticker")}, exchange: {stock.get("exchange")}')
        driver.get(f'https://wallmine.com/{stock.get("exchange")}/{stock.get("stock_ticker")}')
        time.sleep(3)
        company_name = driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/div[2]/a').text
        current_price = driver.find_element_by_xpath('/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[1]/span[1]').text
        percentage = driver.find_element_by_xpath('/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[2]/div').text
        price_movement = True
        if driver.find_elements_by_class_name('badge.badge-success'):
            price_up = driver.find_elements_by_class_name('badge.badge-success')[0].text
            if price_up == percentage:
                print('Price is going up.')
        elif driver.find_elements_by_class_name('badge.badge-danger'):
            price_down = driver.find_elements_by_class_name('badge.badge-danger')[0].text
            if price_down == percentage:
                price_movement = False
                print('Price is going down.')
        amount_changed = float(driver.find_element_by_xpath('/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[1]/span[2]').text[1:])
        market_cap = change_to_float('/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[1]/td/span')
        enterprise_value = change_to_float('/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[2]/td/span')
        revenue = change_to_float('/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[1]/td/span')
        ebitda = change_to_float('/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[2]/td/span')
        income = change_to_float('/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[3]/td/span')
        volume_values = driver.find_element_by_xpath('/html/body/main/section/div[4]/div[1]/div[2]/div[2]/div[1]/table/tbody/tr[1]/td').text.split(' / ')
        volume_purchased = change_to_float(volume_values[0])
        volume_outstanding = change_to_float(volume_values[1])
        stock_object = {
            'company_name': company_name,
            'current_price': current_price,
            'percentage': percentage,
            'price_movement': price_movement,
            'amount_changed': amount_changed,
            'market_cap': market_cap,
            'enterprise_value': enterprise_value,
            'revenue': revenue,
            'ebitda': ebitda,
            'income': income,
            'volume_purchased': volume_purchased,
            'volume_outstanding': volume_outstanding
        }
        print('Here\'s the object:')
        print(stock_object)

In [91]:
from pymongo import MongoClient

client = MongoClient()
db = client['stock_data']
collection = db['current_data']


0: AAPL, exchange: NASDAQ
Price is going up.
Here's the object:
{'company_name': 'Apple Inc', 'current_price': '$127.83', 'percentage': '0.54%', 'price_movement': True, 'amount_changed': 0.69, 'market_cap': None, 'enterprise_value': None, 'revenue': 274515000000.0, 'ebitda': 77344000000.0, 'income': 57411000000.0, 'volume_purchased': None, 'volume_outstanding': 99692000.0}
1: MSFT, exchange: NASDAQ
Price is going up.
Here's the object:
{'company_name': 'Microsoft Corporation', 'current_price': '$216.44', 'percentage': '1.78%', 'price_movement': True, 'amount_changed': 3.79, 'market_cap': None, 'enterprise_value': None, 'revenue': 147114000000.0, 'ebitda': 55823000000.0, 'income': 47496000000.0, 'volume_purchased': 25207000.0, 'volume_outstanding': 24952000.0}
2: AMZN, exchange: NASDAQ
Price is going up.
Here's the object:
{'company_name': 'Amazon.com Inc.', 'current_price': '$3,135.00', 'percentage': '0.99%', 'price_movement': True, 'amount_changed': 30.75, 'market_cap': None, 'enterpr

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[2]/td/span"}
  (Session info: chrome=87.0.4280.141)


In [92]:
from pymongo import MongoClient

client = MongoClient()
db = client['stock_data']
collection = db['current_data']

db.current_data.insert(stock_object)
db.current_data.find_one()

ModuleNotFoundError: No module named 'pymongo'